In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
obf_url = 'https://bulbapedia.bulbagarden.net/wiki/Obsidian_Flames_(TCG)'
obf_page = requests.get(obf_url)

In [4]:
obf_soup = BeautifulSoup(obf_page.text, 'html')

In [5]:
# Incidentally, there is another table with the same formatting.
obf_table = obf_soup.find_all('table', class_ = 'roundy', style = 'float:left; background:#3d0023; border: 3px solid #ed324b')[1]

In [9]:
#Creating Table Headers by specifying the top row.
obf_headers = obf_table.find('tr', style = 'background:#f3983c;')

In [14]:
#Strip the HTML tags for a List of column titles. Removing unnecessary column titles for this project.
obf_column_titles = [header.text.strip() for header in obf_headers.find_all('th')]
to_remove = ['Type', 'Mark', 'Promotion']
for title in to_remove:
    while title in obf_column_titles:
        obf_column_titles.remove(title)

In [16]:
#Create the dataframe with columns set.
obf_df = pd.DataFrame(columns = obf_column_titles)

In [18]:
#Incidentally, the type of the card is tagged as th, not td. It's unneeded for the purposes of this dataframe however.
obf_column_data = obf_table.find_all('td', style = 'background:#FFFFFF')

In [20]:
#Populate dataframe with column data. {'1': 'No.', '2': Mark, '3': 'Card name', '4': 'Rarity'}
column_number = 0
current_entry = []

for value in obf_column_data:
    row_to_update = len(obf_df)
    column_number += 1

    if column_number == 1:
        current_entry.append(value.text.strip())

    #Column 2, the Mark, is unnecessary so it's skipped.

    elif column_number == 3:
        #The ex (and other tags such as VStar) will not show up without additional code.
        card_name = value.text.strip()

        try:
            pokemon_tag = value.find_all('a')[1].get('title')
            card_name += ' ' + pokemon_tag
            current_entry.append(card_name)
        except:
            current_entry.append(card_name)

    elif column_number == 4:
        current_entry.append(value.find('a').get('title'))
        obf_df.loc[row_to_update] = current_entry

        #Reset for the next row.
        current_entry = []
        column_number = 0

In [23]:
obf_df.to_csv(r'C:/Users/zsere/Documents/Raw Data/Pokemon Set Card Info/Obsidian Flames Card Info.csv')